Used to search for files that match a specified pattern.

In [1]:
!pip install pandas
!pip3 install scipy
import pandas as pd
import numpy as np
import glob
import scipy.stats as stats

zsh:1: /opt/homebrew/bin/pip: bad interpreter: /opt/homebrew/opt/python@3.9/bin/python3.9: no such file or directory

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


Xiao Hei Cleaning


In [2]:
# Load the qrels file
qrels_file_path = 'qrels.trec8.adhoc.csv'
qrels_df = pd.read_csv(qrels_file_path, sep=' ', header=None, names=['query_id', 'ignore', 'doc_id', 'relevance'])

#show qrels_dataframe
qrels_df.head()
print(qrels_df.columns)

Index(['query_id', 'ignore', 'doc_id', 'relevance'], dtype='object')


In [3]:
# Get a list of all CSV files in the input directory
input_files = glob.glob('input/*.csv')

# Create an empty list to store the input files dataframes
input_dfs = []

# Loop through each input CSV file
for filename in input_files:
    # Read the input CSV file into a dataframe
    df = pd.read_csv(filename, sep='\t', header=None)
    # Append the dataframe to the list of dataframes
    input_dfs.append(df)
    
raw_data = pd.concat(input_dfs, ignore_index=True)
print('Shape of raw_data:', raw_data.shape)

############################################## Data cleaning ##############################################

# Loop through each input dataframe and filter out rows where does not meet the clean up requirement
# 1. data_id  is not present in qrels_data_ids
# 2. relevant_score equal to 0
# 3. duplicate ranking

# Extract the valid values from the third column of qrels_df
qrels_data_ids = set(qrels_df.iloc[:, 2])

cleaned_dfs = []

for df in input_dfs:
    # # # Get rows where the data id is present in the qrels data ids
    # # df_with_present_data_id = df.iloc[:, 2].isin(qrels_data_ids)
    
    # # # Get rows where relevant_score does not equal 0
    # # df_with_relevant_score_not_zero = df.iloc[:, 4] != 0
    
    # # Combine targeted rows for first and second rule
    # cleaned_df = df[df_with_present_data_id & df_with_relevant_score_not_zero]

    # Get rows where ranking is not duplicated
    cleaned_df = df
    cleaned_dfs.append(cleaned_df)

# Print the shape of the cleaned data
print("this is input data:", cleaned_dfs)

Shape of raw_data: (689266, 6)
this is input data: [        0   1            2    3      4         5
0     401  Q0   FT922-9093    1  25000  READWARE
1     401  Q0   FT924-4344    2  24999  READWARE
2     401  Q0   FT924-5091    3  24998  READWARE
3     401  Q0   FT924-5839    4  24997  READWARE
4     401  Q0   FT924-7558    5  24996  READWARE
...   ...  ..          ...  ...    ...       ...
3055  450  Q0   FT943-9994  159  13986  READWARE
3056  450  Q0  FBIS3-33777  160  13000  READWARE
3057  450  Q0  FBIS3-52235  161  12000  READWARE
3058  450  Q0  FBIS4-12807  162  11000  READWARE
3059  450  Q0  FBIS4-37770  163  10999  READWARE

[3060 rows x 6 columns],          0   1            2     3         4       5
0      401  Q0   FT911-1796     1  0.461869  INQ601
1      401  Q0  FBIS4-55036     2  0.446217  INQ601
2      401  Q0  FBIS3-18916     3  0.444678  INQ601
3      401  Q0  FBIS3-18833     4  0.442222  INQ601
4      401  Q0  FBIS3-17077     5  0.441394  INQ601
...    ...  ..        

In [4]:
# Define the column names
columns = ['query_id', 'ignore', 'doc_id', 'rank', 'score', 'ignore2']

# Create an empty list to store the cleaned dataframes
cleaned_dfs_with_names = []

# Iterate over each cleaned dataframe in cleaned_dfs
for df in cleaned_dfs:
    # Rename the columns
    df = df.rename(columns=dict(enumerate(columns)))
    # Append the renamed dataframe to the new list
    cleaned_dfs_with_names.append(df)

In [5]:
print(cleaned_dfs_with_names)

[      query_id ignore       doc_id  rank  score   ignore2
0          401     Q0   FT922-9093     1  25000  READWARE
1          401     Q0   FT924-4344     2  24999  READWARE
2          401     Q0   FT924-5091     3  24998  READWARE
3          401     Q0   FT924-5839     4  24997  READWARE
4          401     Q0   FT924-7558     5  24996  READWARE
...        ...    ...          ...   ...    ...       ...
3055       450     Q0   FT943-9994   159  13986  READWARE
3056       450     Q0  FBIS3-33777   160  13000  READWARE
3057       450     Q0  FBIS3-52235   161  12000  READWARE
3058       450     Q0  FBIS4-12807   162  11000  READWARE
3059       450     Q0  FBIS4-37770   163  10999  READWARE

[3060 rows x 6 columns],        query_id ignore       doc_id  rank     score ignore2
0           401     Q0   FT911-1796     1  0.461869  INQ601
1           401     Q0  FBIS4-55036     2  0.446217  INQ601
2           401     Q0  FBIS3-18916     3  0.444678  INQ601
3           401     Q0  FBIS3-18833  

In [6]:
# loop over the dataframes and print the first and third columns
for i, cleaned_df in enumerate(cleaned_dfs_with_names):
    print(f"Columns of dataframe {i+1}:")
    print("First column:", cleaned_df.iloc[:, 0])
    print("Third column:", cleaned_df.iloc[:, 2])

Columns of dataframe 1:
First column: 0       401
1       401
2       401
3       401
4       401
       ... 
3055    450
3056    450
3057    450
3058    450
3059    450
Name: query_id, Length: 3060, dtype: int64
Third column: 0        FT922-9093
1        FT924-4344
2        FT924-5091
3        FT924-5839
4        FT924-7558
           ...     
3055     FT943-9994
3056    FBIS3-33777
3057    FBIS3-52235
3058    FBIS4-12807
3059    FBIS4-37770
Name: doc_id, Length: 3060, dtype: object
Columns of dataframe 2:
First column: 0        401
1        401
2        401
3        401
4        401
        ... 
49995    450
49996    450
49997    450
49998    450
49999    450
Name: query_id, Length: 50000, dtype: int64
Third column: 0         FT911-1796
1        FBIS4-55036
2        FBIS3-18916
3        FBIS3-18833
4        FBIS3-17077
            ...     
49995     FT934-2310
49996    FBIS3-56284
49997      FT944-547
49998    FBIS3-54144
49999    FBIS4-37525
Name: doc_id, Length: 50000, dtype: objec

Data Analysis Part 

In [7]:
merged_df = []
for df in cleaned_dfs_with_names:
    cleaned_df_with_qrel = pd.merge(df, qrels_df, on=['query_id','doc_id'], how='left')
    cleaned_df_with_qrel['relevance'] = cleaned_df_with_qrel['relevance'].fillna(0)
    merged_df.append(cleaned_df_with_qrel)
    print(cleaned_df_with_qrel.shape)

(3060, 8)
(50000, 8)
(50000, 8)
(50000, 8)
(49065, 8)
(50000, 8)
(50000, 8)
(50000, 8)
(37141, 8)
(50000, 8)
(50000, 8)
(50000, 8)
(50000, 8)
(50000, 8)
(50000, 8)


In [8]:
print(merged_df)

[      query_id ignore_x       doc_id  rank  score   ignore2  ignore_y   
0          401       Q0   FT922-9093     1  25000  READWARE       0.0  \
1          401       Q0   FT924-4344     2  24999  READWARE       0.0   
2          401       Q0   FT924-5091     3  24998  READWARE       0.0   
3          401       Q0   FT924-5839     4  24997  READWARE       0.0   
4          401       Q0   FT924-7558     5  24996  READWARE       0.0   
...        ...      ...          ...   ...    ...       ...       ...   
3055       450       Q0   FT943-9994   159  13986  READWARE       0.0   
3056       450       Q0  FBIS3-33777   160  13000  READWARE       0.0   
3057       450       Q0  FBIS3-52235   161  12000  READWARE       0.0   
3058       450       Q0  FBIS4-12807   162  11000  READWARE       0.0   
3059       450       Q0  FBIS4-37770   163  10999  READWARE       0.0   

      relevance  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
...         ..

In [9]:
calculate_metrics_precisions = []
calculate_metrics_avg_precisions = []
temp = 0

for df in merged_df:
    freq = df['relevance'].value_counts()[1]
    temp = temp + freq

for index, df in enumerate(merged_df):
    # group by query_id
    grouped = df.groupby('query_id')

    # calculate precision for top 10
    precisions = grouped.apply(lambda x: np.sum(x['relevance'].head(10)) / 10)
    calculate_metrics_precisions.append(precisions)

    # calculate average precision
    avg_precisions = grouped.apply(lambda x: np.nan_to_num(np.sum(x['relevance'] * np.cumsum(x['relevance']) / np.arange(1, len(x['relevance']) + 1)) / np.sum(x['relevance']), nan=0.00000))
    calculate_metrics_avg_precisions.append(avg_precisions)


/var/folders/l3/_ymtxc09315_4jphjq8d1whr0000gn/T/ipykernel_82553/1445219124.py:18: RuntimeWarning: invalid value encountered in scalar divide
  avg_precisions = grouped.apply(lambda x: np.nan_to_num(np.sum(x['relevance'] * np.cumsum(x['relevance']) / np.arange(1, len(x['relevance']) + 1)) / np.sum(x['relevance']), nan=0.00000))


calculate_metrics_precisions = []
calculate_metrics_avg_precisions = []
for df in merged_df:
    # group by query_id
    grouped = df.groupby('query_id')

    # calculate precision for top 10
    precisions = grouped.apply(lambda x: np.sum(x['relevance'].head(10)) / 10)
    calculate_metrics_precisions.append(precisions)

    # calculate average precision
    avg_precisions = grouped.apply(lambda x: np.sum(x['relevance'] * np.cumsum(x['relevance']) / np.arange(1, len(x['relevance']) + 1)) / np.sum(x['relevance']))
    calculate_metrics_avg_precisions.append(avg_precisions)


In [10]:
print("calculate_metrics_precisions",calculate_metrics_precisions)
print("calculate_metrics_avg_pr",calculate_metrics_avg_precisions)

calculate_metrics_precisions [query_id
401    0.8
402    0.7
403    0.8
404    0.8
405    0.7
406    0.6
407    1.0
408    1.0
409    0.9
410    1.0
411    0.9
412    1.0
413    0.9
414    0.8
415    0.6
416    1.0
417    0.9
418    1.0
419    0.7
420    0.8
421    0.7
422    1.0
423    0.8
424    0.9
425    1.0
426    1.0
427    1.0
428    0.9
429    0.8
430    0.2
431    0.9
432    0.1
433    0.6
434    1.0
435    0.7
436    1.0
437    0.4
438    0.9
439    0.7
440    0.6
441    0.6
442    0.7
443    0.3
444    0.4
445    1.0
446    0.9
447    1.0
448    0.3
449    0.6
450    0.8
dtype: float64, query_id
401    0.1
402    0.3
403    0.8
404    0.2
405    0.1
406    0.3
407    0.7
408    0.2
409    0.3
410    1.0
411    0.5
412    0.6
413    0.0
414    0.5
415    1.0
416    0.3
417    0.3
418    0.6
419    0.2
420    0.7
421    0.0
422    0.9
423    0.9
424    0.0
425    1.0
426    0.3
427    0.5
428    0.6
429    0.3
430    0.3
431    0.5
432    0.0
433    0.1
434    0.7
435    0.5
4

In [11]:
# assuming the lists have the same length
precision10_results_df = pd.concat([pd.DataFrame(calculate_metrics_precisions).T],  axis=1)
precision10_results_df.columns = ["Precision@10_" + str(i) for i in range(15)] 

# calculate the overall precision@10
overall_precision = precision10_results_df.mean(axis=1)

# add the new column to the DataFrame
precision10_results_df['Overall Precision@10'] = overall_precision

# calculate the overall row precision@10
overall_precision_row = precision10_results_df.mean(axis=0)

# add the new row to the DataFrame
precision10_results_df.loc["Overall"] = overall_precision_row

# show the new table
print(precision10_results_df)
precision10_results_df.to_csv('resultsT10.csv')

          Precision@10_0  Precision@10_1  Precision@10_2  Precision@10_3   
query_id                                                                   
401                0.800           0.100           0.200             0.1  \
402                0.700           0.300           0.500             0.7   
403                0.800           0.800           0.600             0.9   
404                0.800           0.200           0.200             0.1   
405                0.700           0.100           0.300             0.3   
406                0.600           0.300           0.500             0.3   
407                1.000           0.700           0.800             0.6   
408                1.000           0.200           0.300             0.4   
409                0.900           0.300           0.100             0.2   
410                1.000           1.000           1.000             1.0   
411                0.900           0.500           0.400             0.4   
412         

In [12]:
########################## Question b ##########################
map_results_df = pd.concat([pd.DataFrame(calculate_metrics_avg_precisions).T],  axis=1)
map_results_df.columns = ["MAP_" + str(i) for i in range(15)] 


# calculate the overall precision@10
overall_precision = map_results_df.mean(axis=1)

# add the new column to the DataFrame
map_results_df['Overall Precision@10'] = overall_precision

# calculate the overall row precision@10
overall_precision_row = map_results_df.mean(axis=0)

# add the new row to the DataFrame
map_results_df.loc["Overall"] = overall_precision_row

# show the new table
print(map_results_df)
map_results_df.to_csv('resultsMAP.csv')

             MAP_0     MAP_1     MAP_2     MAP_3     MAP_4     MAP_5   
query_id                                                               
401       0.886494  0.089114  0.133187  0.109808  0.022466  0.088762  \
402       0.757474  0.173972  0.240247  0.439844  0.052108  0.467964   
403       0.850590  0.741483  0.737960  0.751957  0.387609  0.872069   
404       0.843399  0.181481  0.206948  0.228738  0.000000  0.293555   
405       0.777567  0.167936  0.164085  0.200206  0.001091  0.148245   
406       0.656257  0.387500  0.438768  0.231931  0.000000  0.408955   
407       0.830961  0.421762  0.502876  0.305915  0.567553  0.452851   
408       0.932891  0.289035  0.242085  0.354307  0.327780  0.349429   
409       0.792572  0.262796  0.050198  0.223564  0.004049  0.198247   
410       0.623125  0.910139  0.874088  0.749877  0.000000  0.783561   
411       0.919772  0.295043  0.290586  0.417404  0.000000  0.562747   
412       0.748386  0.391078  0.577200  0.070499  0.020216  0.58

In [13]:
########################## Question c ##########################

# Retrieve the precision@10 values for each system, excluding the last column value
list_system_precision10 = precision10_results_df.loc["Overall"][:-1].values

# Retrieve the mean average precision (MAP) values for each system, excluding the last column value
list_system_map = map_results_df.loc["Overall"][:-1].values

# Calculate the correlation coefficient between precision@10 and MAP values
correlation_coefficient = np.corrcoef(list_system_precision10, list_system_map)[0, 1]

# Print the correlation coefficient with an explanation
print("Correlation Coefficient: {:.2f}".format(correlation_coefficient))
print("The correlation coefficient indicates the strength and direction of the linear relationship between precision@10 and MAP values.")
print("A value close to 1 suggests a strong positive correlation, while a value close to -1 suggests a strong negative correlation.")
print("A value close to 0 indicates a weak or no correlation between the two variables.")

Correlation Coefficient: 0.91
The correlation coefficient indicates the strength and direction of the linear relationship between precision@10 and MAP values.
A value close to 1 suggests a strong positive correlation, while a value close to -1 suggests a strong negative correlation.
A value close to 0 indicates a weak or no correlation between the two variables.


In [23]:
import scipy.stats as stats

significant_pairs = 0  # Counter for significant pairs

for i in range(len(list_system_map)):
    for j in range(i+1, len(list_system_map)):
        system1 = list_system_map[i]
        system2 = list_system_map[j]

        # Perform the Mann-Whitney U test between system1 and system2
        u_statistic, p_value = stats.mannwhitneyu(system1, system2, alternative='two-sided')
        
        if p_value < 0.05:
            significant_pairs += 1

        # Print the result with an explanation
        print("Significance testing between System {} and System {}: p-value = {:.4f}".format(i+1, j+1, p_value))

print("\n\nNumber of significantly different pairs: {}".format(significant_pairs))

Significance testing between System 1 and System 2: p-value = 1.0000
Significance testing between System 1 and System 3: p-value = 1.0000
Significance testing between System 1 and System 4: p-value = 1.0000
Significance testing between System 1 and System 5: p-value = 1.0000
Significance testing between System 1 and System 6: p-value = 1.0000
Significance testing between System 1 and System 7: p-value = 1.0000
Significance testing between System 1 and System 8: p-value = 1.0000
Significance testing between System 1 and System 9: p-value = 1.0000
Significance testing between System 1 and System 10: p-value = 1.0000
Significance testing between System 1 and System 11: p-value = 1.0000
Significance testing between System 1 and System 12: p-value = 1.0000
Significance testing between System 1 and System 13: p-value = 1.0000
Significance testing between System 1 and System 14: p-value = 1.0000
Significance testing between System 1 and System 15: p-value = 1.0000
Significance testing between